# PyTorch之小试牛刀
## 1 PyTorch的核心是两个主要特征：

* 一个n维张量，类似于numpy，但可以在GPU上运行
* 搭建和训练神经网络时的自动微分/求导机制

本章节我们将使用全连接的ReLU网络作为运行示例。该网络将有一个单一的隐藏层，并将使用梯度下降训练，通过最小化网络输出和真正 结果的欧几里得距离，来拟合随机生成的数据。

## 2.张量
### 2.1 热身: Numpy

在介绍PyTorch之前，本章节将首先使用numpy实现网络。 Numpy提供了一个n维数组对象，以及许多用于操作这些数组的 函数。Numpy是用于科学计算的通用框架;它对计算图、深度学习和梯度一无所知。然而，我们可以很容易地使用NumPy，手动实现网络的 前向和反向传播，来拟合随机数据：

In [1]:
# -*- coding: utf-8 -*-
import numpy as np

# N是批量大小; D_in是输入维度;
# 49/5000 H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机输入和输出数据
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# 随机初始化权重
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    # 前向传递：计算预测值y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # 计算和打印损失loss
    loss = np.square(y_pred - y).sum()
    print(t, loss)

    # 反向传播，计算w1和w2对loss的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # 更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 27930962.626591586
1 23238376.280001074
2 21675841.059636045
3 20378262.33716751
4 17876371.56071773
5 14194628.788396485
6 10153716.33880689
7 6753356.41316016
8 4335834.205835767
9 2806761.593592204
10 1886341.613054935
11 1339614.1242764802
12 1006156.305051334
13 793220.3674432214
14 648780.2419889863
15 544755.5943501391
16 465777.6725268854
17 403381.3898074983
18 352394.68060757406
19 309839.96101274394
20 273824.5127531792
21 243016.56023985596
22 216449.02905192954
23 193362.51360237348
24 173214.27229573944
25 155558.60381408996
26 140048.57648860713
27 126359.28873695126
28 114272.90923586207
29 103553.22658352496
30 94012.77031522106
31 85493.84394913545
32 77871.55197040059
33 71041.2692865612
34 64904.75719916748
35 59379.11953487797
36 54398.08054644785
37 49894.43135816626
38 45817.48739701168
39 42122.073324273864
40 38766.90148103269
41 35718.44462937197
42 32941.39910628731
43 30409.08964016982
44 28097.387045903633
45 25983.509334494644
46 24049.163773629858
47 22

445 0.0002897471518665945
446 0.00027862542929176897
447 0.00026793112602672763
448 0.00025764674277496344
449 0.0002477588647167707
450 0.00023825097094874652
451 0.0002291103388496767
452 0.00022032133698574177
453 0.00021186937609736145
454 0.00020374296739280698
455 0.0001959301274880925
456 0.0001884162510046456
457 0.00018119213492709518
458 0.00017424449934890754
459 0.0001675642045541582
460 0.00016114018513041788
461 0.00015496320323202804
462 0.00014902474837056977
463 0.0001433141214667513
464 0.0001378223624026466
465 0.0001325413947311626
466 0.00012746300691186675
467 0.00012258002315269526
468 0.00011788515345915348
469 0.00011336979305557448
470 0.00010902797312898915
471 0.00010485261734774992
472 0.00010083741341660615
473 9.697706843661666e-05
474 9.326416200109079e-05
475 8.96944708584908e-05
476 8.626176785894335e-05
477 8.295982296669357e-05
478 7.978494700011375e-05
479 7.673185917834008e-05
480 7.379546665327095e-05
481 7.097181163612289e-05
482 6.82564863350418

### 2.2 PyTorch：张量

Numpy是一个很棒的框架，但它不能利用GPU来加速其数值计算。 对于现代深度神经网络，GPU通常提供50倍或更高的加速，所以，numpy不能满足当代深度学习的需求。

在这里，先介绍最基本的PyTorch概念：

张量（Tensor）：PyTorch的tensor在概念上与numpy的array相同： tensor是一个n维数组，PyTorch提供了许多函数用于操作这些张量。任何希望使用NumPy执行的计算也可以使用PyTorch的tensor来完成，可以认为它们是科学计算的通用工具。

与Numpy不同，PyTorch可以利用GPU加速其数值计算。要在GPU上运行Tensor,在构造张量使用device参数把tensor建立在GPU上。

在这里，本章使用tensors将随机数据上训练一个两层的网络。和前面NumPy的例子类似，我们使用PyTorch的tensor，手动在网络中实现前向传播和反向传播：

In [2]:
# -*- coding: utf-8 -*-

import torch


dtype = torch.float
device = torch.device("cpu")
# device = torch.device（“cuda：0”）＃取消注释以在GPU上运行

# N是批量大小; D_in是输入维度;
# H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

#创建随机输入和输出数据
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# 随机初始化权重
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 前向传递：计算预测y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # 计算和打印损失
    loss = (y_pred - y).pow(2).sum().item()
    print(t, loss)

    # Backprop计算w1和w2相对于损耗的梯度
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # 使用梯度下降更新权重
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

0 44385500.0
1 48499220.0
2 52533568.0
3 43460428.0
4 24605720.0
5 10028097.0
6 4121184.25
7 2223578.25
8 1545717.0
9 1209791.0
10 990661.125
11 826921.25
12 697882.0
13 593703.3125
14 508633.4375
15 438422.0625
16 380033.8125
17 331041.03125
18 289654.6875
19 254497.640625
20 224467.265625
21 198686.25
22 176441.96875
23 157145.78125
24 140350.71875
25 125671.9140625
26 112819.7578125
27 101520.71875
28 91542.8359375
29 82671.5703125
30 74801.9921875
31 67804.9140625
32 61561.92578125
33 55978.0078125
34 50981.6484375
35 46498.6015625
36 42466.234375
37 38831.0703125
38 35550.08203125
39 32596.38671875
40 29920.6171875
41 27493.689453125
42 25288.310546875
43 23281.71875
44 21455.271484375
45 19791.5078125
46 18272.38671875
47 16884.34765625
48 15613.4755859375
49 14449.6025390625
50 13382.57421875
51 12403.2802734375
52 11504.1044921875
53 10674.734375
54 9911.6513671875
55 9208.82421875
56 8563.6767578125
57 7969.0224609375
58 7419.52099609375
59 6911.7236328125
60 6442.373046875
61

403 0.0015879161655902863
404 0.0015389741165563464
405 0.0014902327675372362
406 0.0014450997114181519
407 0.001398430671542883
408 0.001357698580250144
409 0.0013169324956834316
410 0.001275405054911971
411 0.0012391724158078432
412 0.0012023333692923188
413 0.0011657802388072014
414 0.0011332747526466846
415 0.0010986441047862172
416 0.0010656046215444803
417 0.001035052235238254
418 0.0010040969355031848
419 0.0009764854912646115
420 0.0009470322402194142
421 0.0009203327354043722
422 0.0008944519213400781
423 0.0008689028909429908
424 0.0008443623082712293
425 0.0008205524645745754
426 0.0007979755755513906
427 0.0007751614321023226
428 0.0007533773896284401
429 0.0007332595996558666
430 0.0007141301175579429
431 0.0006951683317311108
432 0.0006761839031241834
433 0.0006591627607122064
434 0.000641460414044559
435 0.0006242170929908752
436 0.0006080992170609534
437 0.000591993099078536
438 0.0005761150387115777
439 0.0005627940990962088
440 0.000547942123375833
441 0.0005334002198

## 3.自动求导
### 3.1 PyTorch：张量和自动求导

在上面的例子中，需要手动实现神经网络的前向和后向 传递。手动实现反向传递对于小型双层网络来说并不是什么大问 题，但对于大型复杂网络来说很快就会变得非常繁琐。

但是可以使用自动微分来自动计算神经网络中的后向传递。 PyTorch中的 autograd 包提供了这个功能。当使用autograd时，网络前向传播将定义一个计算图；图中的节点是tensor，边是函数， 这些函数是输出tensor到输入tensor的映射。这张计算图使得在网络中反向传播时梯度的计算十分简单。

这听起来很复杂，在实践中使用起来非常简单。 如果我们想计算某些的tensor的梯度，我们只需要在建立这个tensor时加入这么一句：requires_grad=True。这个tensor上的任何PyTorch的操作都将构造一个计算图，从而允许我们稍后在图中执行反向传播。如果这个tensor x的requires_grad=True，那么反向传播之后x.grad将会是另一个张量，其为x关于某个标量值的梯度。

有时可能希望防止PyTorch在requires_grad=True的张量执行某些操作时构建计算图；例如，在训练神经网络时，我们通常不希望通过权重更新步骤进行反向传播。在这种情况下，我们可以使用torch.no_grad()上下文管理器来防止构造计算图。

下面我们使用PyTorch的Tensors和autograd来实现我们的两层的神经网络；我们不再需要手动执行网络的反向传播：

In [7]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
#  device = torch.device("cpu")
device = torch.device("cuda")# 取消注释以在GPU上运行

# N是批量大小; D_in是输入维度;
# H是隐藏的维度; D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 创建随机Tensors以保持输入和输出。
# 设置requires_grad = False表示我们不需要计算渐变
# 在向后传球期间对于这些Tensors。
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)


# 为权重创建随机Tensors。
# 设置requires_grad = True表示我们想要计算渐变
# 在向后传球期间尊重这些张贴。
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 前向传播：使用tensors上的操作计算预测值y; 
      # 由于w1和w2有requires_grad=True，涉及这些张量的操作将让PyTorch构建计算图，
    # 从而允许自动计算梯度。由于我们不再手工实现反向传播，所以不需要保留中间值的引用。
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # 使用Tensors上的操作计算和打印丢失。
    # loss是一个形状为()的张量
    # loss.item() 得到这个张量对应的python数值
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    # 使用autograd计算反向传播。这个调用将计算loss对所有requires_grad=True的tensor的梯度。
    # 这次调用后，w1.grad和w2.grad将分别是loss对w1和w2的梯度张量。
    loss.backward()
    
    # 使用梯度下降更新权重。对于这一步，我们只想对w1和w2的值进行原地改变；不想为更新阶段构建计算图，
    # 所以我们使用torch.no_grad()上下文管理器防止PyTorch为更新构建计算图
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 反向传播后手动将梯度设置为零
        w1.grad.zero_()
        w2.grad.zero_()

0 33072460.0
1 36317924.0
2 43473440.0
3 45952808.0
4 36844436.0
5 21113208.0
6 9274054.0
7 3926198.25
8 1996033.75
9 1289016.25
10 972018.125
11 788021.625
12 658973.6875
13 559203.0
14 478637.875
15 412263.6875
16 357043.375
17 310816.5625
18 271704.9375
19 238453.21875
20 210014.765625
21 185588.53125
22 164499.25
23 146213.265625
24 130282.296875
25 116384.2734375
26 104213.765625
27 93522.484375
28 84109.375
29 75789.0625
30 68419.421875
31 61871.48046875
32 56051.203125
33 50862.34375
34 46226.9921875
35 42077.21875
36 38350.5
37 34997.6875
38 31977.1953125
39 29253.109375
40 26790.9453125
41 24564.302734375
42 22544.55078125
43 20709.78515625
44 19042.22265625
45 17524.4453125
46 16143.154296875
47 14882.8271484375
48 13731.810546875
49 12679.248046875
50 11715.92578125
51 10833.55859375
52 10024.33203125
53 9282.05078125
54 8600.482421875
55 7973.9521484375
56 7397.3876953125
57 6866.6845703125
58 6377.4072265625
59 5926.48681640625
60 5510.0703125
61 5125.490234375
62 4770.185

375 0.0004199991817586124
376 0.0004068335110787302
377 0.0003953156992793083
378 0.00038335658609867096
379 0.00037175690522417426
380 0.0003609236446209252
381 0.00035027810372412205
382 0.00034052430419251323
383 0.0003304479760117829
384 0.0003208898997399956
385 0.00031284004217013717
386 0.0003035904956050217
387 0.0002966442843899131
388 0.0002885141293518245
389 0.00028000620659440756
390 0.0002727047831285745
391 0.0002658597950357944
392 0.0002585647162050009
393 0.00025191440363414586
394 0.00024509726790711284
395 0.00023963373678270727
396 0.00023406607215292752
397 0.00022791163064539433
398 0.0002216279972344637
399 0.0002161123265977949
400 0.00021072784147690982
401 0.00020585664606187493
402 0.00020117926760576665
403 0.000196715714992024
404 0.00019185495330020785
405 0.0001870517444331199
406 0.00018275162437930703
407 0.00017868215218186378
408 0.00017450886662118137
409 0.00017079105600714684
410 0.0001669218618189916
411 0.00016328771016560495
412 0.0001598586095

### 3.2 PyTorch：定义新的自动求导函数

在底层，每一个原始的自动求导运算实际上是两个在Tensor上运行的函数。其中，forward函数计算从输入Tensors获得的输出Tensors。而backward函数接收输出Tensors对于某个标量值的梯度，并且计算输入Tensors相对于该相同标量值的梯度。

在PyTorch中，我们可以很容易地通过定义torch.autograd.Function的子类并实现forward和backward函数，来定义自己的自动求导运算。之后我们就可以使用这个新的自动梯度运算符了。然后，我们可以通过构造一个实例并像调用函数一样，传入包含输入数据的tensor调用它，这样来使用新的自动求导运算。

这个例子中，我们自定义一个自动求导函数来展示ReLU的非线性。并用它实现我们的两层网络：

In [8]:
import torch

class MyReLU(torch.autograd.Function):
    """
    我们可以通过建立torch.autograd的子类来实现我们自定义的autograd函数，
    并完成张量的正向和反向传播。
    """
    @staticmethod
    def forward(ctx, x):
        """
        在正向传播中，我们接收到一个上下文对象和一个包含输入的张量；
        我们必须返回一个包含输出的张量，
        并且我们可以使用上下文对象来缓存对象，以便在反向传播中使用。
        """
        ctx.save_for_backward(x)
        return x.clamp(min=0)

    @staticmethod
    def backward(ctx, grad_output):
        """
        在反向传播中，我们接收到上下文对象和一个张量，
        其包含了相对于正向传播过程中产生的输出的损失的梯度。
        我们可以从上下文对象中检索缓存的数据，
        并且必须计算并返回与正向传播的输入相关的损失的梯度。
        """
        x, = ctx.saved_tensors
        grad_x = grad_output.clone()
        grad_x[x < 0] = 0
        return grad_x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in, device=device)
y = torch.randn(N, D_out, device=device)

# 产生随机权重的张量
w1 = torch.randn(D_in, H, device=device, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, requires_grad=True)

learning_rate = 1e-6
for t in range(500):
    # 正向传播：使用张量上的操作来计算输出值y；
    # 我们通过调用 MyReLU.apply 函数来使用自定义的ReLU
    y_pred = MyReLU.apply(x.mm(w1)).mm(w2)

    # 计算并输出loss
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())

    # 使用autograd计算反向传播过程。
    loss.backward()

    with torch.no_grad():
        # 用梯度下降更新权重
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 在反向传播之后手动清零梯度
        w1.grad.zero_()
        w2.grad.zero_()

0 33350252.0
1 35182280.0
2 48078512.0
3 62840896.0
4 61983056.0
5 37444208.0
6 13570354.0
7 4051736.0
8 1757284.5
9 1163031.75
10 917544.5
11 761343.375
12 642805.5625
13 547824.625
14 470122.75
15 405809.9375
16 352256.65625
17 307211.53125
18 269134.75
19 236737.21875
20 209003.125
21 185136.40625
22 164504.59375
23 146580.625
24 130939.8828125
25 117272.84375
26 105286.6640625
27 94728.96875
28 85410.3671875
29 77155.5546875
30 69829.0390625
31 63306.61328125
32 57483.7109375
33 52275.609375
34 47617.34375
35 43435.765625
36 39671.08984375
37 36277.4765625
38 33215.53125
39 30447.4609375
40 27940.21484375
41 25666.82421875
42 23604.35546875
43 21729.9765625
44 20021.8828125
45 18463.40625
46 17041.12109375
47 15740.611328125
48 14550.890625
49 13461.4814453125
50 12462.9306640625
51 11546.0234375
52 10703.43359375
53 9928.8876953125
54 9216.072265625
55 8559.16796875
56 7953.6015625
57 7394.7841796875
58 6879.0146484375
59 6402.111328125
60 5961.21484375
61 5553.3447265625
62 5175.

380 0.0006802238058298826
381 0.000660230522044003
382 0.0006410215282812715
383 0.0006218809867277741
384 0.0006045515183359385
385 0.0005868251901119947
386 0.0005700060864910483
387 0.0005534289521165192
388 0.0005381993250921369
389 0.0005230279639363289
390 0.0005075554363429546
391 0.0004931339644826949
392 0.0004797751025762409
393 0.0004670039634220302
394 0.00045391032472252846
395 0.0004417148884385824
396 0.0004298936401028186
397 0.00041870574932545424
398 0.000407255778554827
399 0.00039658776950091124
400 0.000386547006200999
401 0.00037651514867320657
402 0.0003668105637189001
403 0.000357535551302135
404 0.00034901208709925413
405 0.0003398590488359332
406 0.00033165927743539214
407 0.0003235884360037744
408 0.00031481331097893417
409 0.00030784946284256876
410 0.00029984675347805023
411 0.000293161952868104
412 0.00028594990726560354
413 0.0002799590292852372
414 0.0002733761502895504
415 0.0002668567467480898
416 0.00026099575916305184
417 0.0002551313955336809
418 0.

### 3.3 TensorFlow：静态图

PyTorch自动求导看起来非常像TensorFlow：这两个框架中，我们都定义计算图，使用自动微分来计算梯度。两者最大的不同就是TensorFlow的计算图是静态的，而PyTorch使用动态的计算图。

在TensorFlow中，我们定义计算图一次，然后重复执行这个相同的图，可能会提供不同的输入数据。而在PyTorch中，每一个前向通道定义一个新的计算图。

静态图的好处在于你可以预先对图进行优化。例如，一个框架可能要融合一些图的运算来提升效率，或者产生一个策略来将图分布到多个GPU或机器上。如果重复使用相同的图，那么在重复运行同一个图时，，前期潜在的代价高昂的预先优化的消耗就会被分摊开。

静态图和动态图的一个区别是控制流。对于一些模型，我们希望对每个数据点执行不同的计算。例如，一个递归神经网络可能对于每个数据点执行不同的时间步数，这个展开（unrolling）可以作为一个循环来实现。对于一个静态图，循环结构要作为图的一部分。因此，TensorFlow提供了运算符（例如tf.scan）来把循环嵌入到图当中。对于动态图来说，情况更加简单：既然我们为每个例子即时创建图，我们可以使用普通的命令式控制流来为每个输入执行不同的计算。

为了与上面的PyTorch自动梯度实例做对比，我们使用TensorFlow来拟合一个简单的2层网络：

In [10]:
import tensorflow as tf
import numpy as np

# 首先我们建立计算图（computational graph）

# N是批大小；D是输入维度；
# H是隐藏层维度；D_out是输出维度。
N, D_in, H, D_out = 64, 1000, 100, 10

# 为输入和目标数据创建placeholder；
# 当执行计算图时，他们将会被真实的数据填充
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

# 为权重创建Variable并用随机数据初始化
# TensorFlow的Variable在执行计算图时不会改变
w1 = tf.Variable(tf.random_normal((D_in, H)))
w2 = tf.Variable(tf.random_normal((H, D_out)))

# 前向传播：使用TensorFlow的张量运算计算预测值y。
# 注意这段代码实际上不执行任何数值运算；
# 它只是建立了我们稍后将执行的计算图。
h = tf.matmul(x, w1)
h_relu = tf.maximum(h, tf.zeros(1))
y_pred = tf.matmul(h_relu, w2)

# 使用TensorFlow的张量运算损失（loss）
loss = tf.reduce_sum((y - y_pred) ** 2.0)

# 计算loss对于w1和w2的导数
grad_w1, grad_w2 = tf.gradients(loss, [w1, w2])

# 使用梯度下降更新权重。为了实际更新权重，我们需要在执行计算图时计算new_w1和new_w2。
# 注意，在TensorFlow中，更新权重值的行为是计算图的一部分;
# 但在PyTorch中，这发生在计算图形之外。
learning_rate = 1e-6
new_w1 = w1.assign(w1 - learning_rate * grad_w1)
new_w2 = w2.assign(w2 - learning_rate * grad_w2)

# 现在我们搭建好了计算图，所以我们开始一个TensorFlow的会话（session）来实际执行计算图。
with tf.Session() as sess:

    # 运行一次计算图来初始化Variable w1和w2
    sess.run(tf.global_variables_initializer())

    # 创建numpy数组来存储输入x和目标y的实际数据
    x_value = np.random.randn(N, D_in)
    y_value = np.random.randn(N, D_out)
    
    for _ in range(500):
        # 多次运行计算图。每次执行时，我们都用feed_dict参数，
        # 将x_value绑定到x，将y_value绑定到y，
        # 每次执行图形时我们都要计算损失、new_w1和new_w2；
        # 这些张量的值以numpy数组的形式返回。
        loss_value, _, _ = sess.run([loss, new_w1, new_w2], 
                                    feed_dict={x: x_value, y: y_value})
        print(loss_value)

37834360.0
34140830.0
30454282.0
23581424.0
15544606.0
8992188.0
5045627.5
2966809.2
1917602.9
1361097.4
1037858.06
829074.56
681349.4
569773.75
482065.97
411268.7
353258.8
305221.75
265034.62
231158.56
202390.36
177795.5
156705.81
138508.84
122754.625
109059.34
97112.42
86656.695
77481.44
69401.5
62274.57
55975.953
50396.508
45446.54
41039.523
37108.734
33597.582
30464.277
27655.66
25133.705
22865.383
20823.402
18982.75
17322.055
15823.227
14466.283
13235.531
12119.052
11104.992
10182.565
9343.815
8579.822
7883.4688
7248.081
6667.6206
6138.454
5654.605
5211.772
4806.4
4434.679
4093.6562
3780.7395
3493.3184
3229.28
2986.5938
2763.2522
2557.6665
2368.3032
2193.849
2033.0178
1884.6902
1747.8367
1621.4861
1504.823
1397.0583
1297.3973
1205.2997
1120.0627
1041.2064
968.2185
900.5759
837.93134
779.8517
725.9915
676.05164
629.66846
586.60974
546.6418
509.50793
475.0216
442.9525
413.14697
385.4439
359.6665
335.6814
313.3634
292.57584
273.22498
255.20149
238.41043
222.77121
208.19495
194.61002


## 4.nn模块
### 4.1 PyTorch：nn

计算图和autograd是十分强大的工具，可以定义复杂的操作并自动求导；然而对于大规模的网络，autograd太过于底层。 在构建神经网络时，我们经常考虑将计算安排成层，其中一些具有可学习的参数，它们将在学习过程中进行优化。

TensorFlow里，有类似Keras，TensorFlow-Slim和TFLearn这种封装了底层计算图的高度抽象的接口，这使得构建网络十分方便。

在PyTorch中，包nn完成了同样的功能。nn包中定义一组大致等价于层的模块。一个模块接受输入的tesnor，计算输出的tensor，而且 还保存了一些内部状态比如需要学习的tensor的参数等。nn包中也定义了一组损失函数（loss functions），用来训练神经网络。

这个例子中，我们用nn包实现两层的网络：

In [13]:
# -*- coding: utf-8 -*-
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

#创建输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包将我们的模型定义为一系列的层。
# nn.Sequential是包含其他模块的模块，并按顺序应用这些模块来产生其输出。
# 每个线性模块使用线性函数从输入计算输出，并保存其内部的权重和偏差张量。
# 在构造模型之后，我们使用.to()方法将其移动到所需的设备。
model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out),
)

# nn包还包含常用的损失函数的定义；
# 在这种情况下，我们将使用平均平方误差(MSE)作为我们的损失函数。
# 设置reduction='sum'，表示我们计算的是平方误差的“和”，而不是平均值;
# 这是为了与前面我们手工计算损失的例子保持一致，
# 但是在实践中，通过设置reduction='elementwise_mean'来使用均方误差作为损失更为常见。
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t in range(500):
    # 前向传播：通过向模型传入x计算预测的y。
    # 模块对象重载了__call__运算符，所以可以像函数那样调用它们。
    # 这么做相当于向模块传入了一个张量，然后它返回了一个输出张量。
    y_pred = model(x)

     # 计算并打印损失。
     # 传递包含y的预测值和真实值的张量，损失函数返回包含损失的张量。
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 反向传播之前清零梯度
    model.zero_grad()

    # 反向传播：计算模型的损失对所有可学习参数的导数（梯度）。
    # 在内部，每个模块的参数存储在requires_grad=True的张量中，
    # 因此这个调用将计算模型中所有可学习参数的梯度。
    loss.backward()

    # 使用梯度下降更新权重。
    # 每个参数都是张量，所以我们可以像我们以前那样可以得到它的数值和梯度
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

0 707.493896484375
1 659.0013427734375
2 616.656005859375
3 579.3424682617188
4 546.095947265625
5 515.9916381835938
6 488.25439453125
7 462.66387939453125
8 438.6698913574219
9 416.0790100097656
10 394.76776123046875
11 374.64556884765625
12 355.58282470703125
13 337.397705078125
14 320.02862548828125
15 303.43682861328125
16 287.5411682128906
17 272.34881591796875
18 257.8196105957031
19 243.97015380859375
20 230.71087646484375
21 218.01414489746094
22 205.8751678466797
23 194.21664428710938
24 183.06248474121094
25 172.4652862548828
26 162.39723205566406
27 152.79644775390625
28 143.6934051513672
29 135.0541229248047
30 126.87467956542969
31 119.11138916015625
32 111.77177429199219
33 104.83367919921875
34 98.28571319580078
35 92.11018371582031
36 86.27810668945312
37 80.78791809082031
38 75.61434173583984
39 70.76011657714844
40 66.20091247558594
41 61.91813659667969
42 57.899471282958984
43 54.14214324951172
44 50.627681732177734
45 47.3504638671875
46 44.293800354003906
47 41.443

356 0.0004033672157675028
357 0.00039367424324154854
358 0.00038422178477048874
359 0.0003750094911083579
360 0.00036602330510504544
361 0.000357263459591195
362 0.00034870856325142086
363 0.00034037401201203465
364 0.0003322442644275725
365 0.00032430639839731157
366 0.00031657013460062444
367 0.00030902394792065024
368 0.0003016602713614702
369 0.00029448376153595746
370 0.0002874737256206572
371 0.0002806438133120537
372 0.0002739716146606952
373 0.0002674648421816528
374 0.0002611208474263549
375 0.00025492964778095484
376 0.00024889069027267396
377 0.0002429956803098321
378 0.00023724525817669928
379 0.00023163242440205067
380 0.00022615326452068985
381 0.00022081130009610206
382 0.00021560171444434673
383 0.00021051369549240917
384 0.00020555226365104318
385 0.00020070371101610363
386 0.00019597288337536156
387 0.00019135861657559872
388 0.00018685664690565318
389 0.00018246317631565034
390 0.00017817242769524455
391 0.00017398432828485966
392 0.00016989918367471546
393 0.0001659

### 4.2 PyTorch：optim

到目前为止，我们已经通过手动改变包含可学习参数的张量来更新模型的权重。对于随机梯度下降(SGD/stochastic gradient descent)等简单的优化算法来说，这不是一个很大的负担，但在实践中，我们经常使用AdaGrad、RMSProp、Adam等更复杂的优化器来训练神经网络。

In [14]:
import torch

# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生随机输入和输出张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 使用nn包定义模型和损失函数
model = torch.nn.Sequential(
          torch.nn.Linear(D_in, H),
          torch.nn.ReLU(),
          torch.nn.Linear(H, D_out),
        )
loss_fn = torch.nn.MSELoss(reduction='sum')

# 使用optim包定义优化器（Optimizer）。Optimizer将会为我们更新模型的权重。
# 这里我们使用Adam优化方法；optim包还包含了许多别的优化算法。
# Adam构造函数的第一个参数告诉优化器应该更新哪些张量。
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(500):

    # 前向传播：通过像模型输入x计算预测的y
    y_pred = model(x)

    # 计算并打印loss
    loss = loss_fn(y_pred, y)
    print(t, loss.item())
    
    # 在反向传播之前，使用optimizer将它要更新的所有张量的梯度清零(这些张量是模型可学习的权重)
    optimizer.zero_grad()

    # 反向传播：根据模型的参数计算loss的梯度
    loss.backward()

    # 调用Optimizer的step函数使它所有参数更新
    optimizer.step()

0 700.3743286132812
1 682.90478515625
2 665.9736328125
3 649.5538940429688
4 633.654541015625
5 618.2203979492188
6 603.2433471679688
7 588.813232421875
8 574.8646240234375
9 561.2676391601562
10 548.0455322265625
11 535.1727294921875
12 522.6282348632812
13 510.3653564453125
14 498.4137878417969
15 486.73590087890625
16 475.3027648925781
17 464.1619567871094
18 453.3089599609375
19 442.7607421875
20 432.5345153808594
21 422.5931091308594
22 412.91845703125
23 403.5364685058594
24 394.4033203125
25 385.5277404785156
26 376.8576965332031
27 368.384521484375
28 360.104248046875
29 352.0248107910156
30 344.10699462890625
31 336.3180236816406
32 328.6804504394531
33 321.2043762207031
34 313.87652587890625
35 306.6881103515625
36 299.6590270996094
37 292.7806091308594
38 286.0653076171875
39 279.49200439453125
40 273.04278564453125
41 266.7403259277344
42 260.5885314941406
43 254.54867553710938
44 248.6282958984375
45 242.8044891357422
46 237.07632446289062
47 231.45590209960938
48 225.9326

378 3.599050978664309e-05
379 3.371841012267396e-05
380 3.158830077154562e-05
381 2.9584160074591637e-05
382 2.7707774279406294e-05
383 2.5941646526916884e-05
384 2.4287704945891164e-05
385 2.2732318029738963e-05
386 2.1278043277561665e-05
387 1.991105091292411e-05
388 1.8629547412274405e-05
389 1.7425221813027747e-05
390 1.6299691196763888e-05
391 1.5242208064591978e-05
392 1.4253982953960076e-05
393 1.332575538981473e-05
394 1.2456558579287957e-05
395 1.1640036973403767e-05
396 1.0878285138460342e-05
397 1.0163782462768722e-05
398 9.495497579337098e-06
399 8.86897396412678e-06
400 8.283049282908905e-06
401 7.733671736787073e-06
402 7.220523002615664e-06
403 6.740241587976925e-06
404 6.289724751695758e-06
405 5.868223070137901e-06
406 5.4754923439759295e-06
407 5.107918241264997e-06
408 4.76438572150073e-06
409 4.442247700353619e-06
410 4.14157466366305e-06
411 3.861520781356376e-06
412 3.5989139632874867e-06
413 3.3536696264491184e-06
414 3.1254312489181757e-06
415 2.9115542474755784

### 4.3 PyTorch：自定义nn模块

有时候需要指定比现有模块序列更复杂的模型；对于这些情况，可以通过继承nn.Module并定义forward函数，这个forward函数可以 使用其他模块或者其他的自动求导运算来接收输入tensor，产生输出tensor。

在这个例子中，我们用自定义Module的子类构建两层网络：

In [15]:
import torch

class TwoLayerNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们实例化了两个nn.Linear模块，并将它们作为成员变量。
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        在前向传播的函数中，我们接收一个输入的张量，也必须返回一个输出张量。
        我们可以使用构造函数中定义的模块以及张量上的任意的（可微分的）操作。
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred

# N是批大小； D_in 是输入维度；
# H 是隐藏层维度； D_out 是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出的随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 通过实例化上面定义的类来构建我们的模型。
model = TwoLayerNet(D_in, H, D_out)

# 构造损失函数和优化器。
# SGD构造函数中对model.parameters()的调用，
# 将包含模型的一部分，即两个nn.Linear模块的可学习参数。
loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
for t in range(500):
    # 前向传播：通过向模型传递x计算预测值y
    y_pred = model(x)

    #计算并输出loss
    loss = loss_fn(y_pred, y)
    print(t, loss.item())

    # 清零梯度，反向传播，更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 645.9310302734375
1 600.5865478515625
2 561.0447998046875
3 526.1351928710938
4 494.9022216796875
5 466.513427734375
6 440.66094970703125
7 417.03704833984375
8 395.1091003417969
9 374.56646728515625
10 355.40869140625
11 337.477294921875
12 320.4334411621094
13 304.4036865234375
14 289.263916015625
15 274.9032287597656
16 261.2231140136719
17 248.1845703125
18 235.7180633544922
19 223.86480712890625
20 212.53050231933594
21 201.70639038085938
22 191.37269592285156
23 181.45816040039062
24 171.9826202392578
25 162.94686889648438
26 154.32229614257812
27 146.07113647460938
28 138.21920776367188
29 130.73683166503906
30 123.61979675292969
31 116.84581756591797
32 110.3930892944336
33 104.25112915039062
34 98.41188049316406
35 92.87798309326172
36 87.63231658935547
37 82.65376281738281
38 77.94119262695312
39 73.48323059082031
40 69.2674789428711
41 65.28206634521484
42 61.517295837402344
43 57.96815490722656
44 54.61868667602539
45 51.4591178894043
46 48.48001480102539
47 45.6697540283

386 0.00016649009194225073
387 0.00016269352636300027
388 0.00015898568381089717
389 0.00015537551371380687
390 0.00015185093798208982
391 0.00014840839139651507
392 0.00014505774015560746
393 0.00014178083802107722
394 0.00013858595048077404
395 0.00013546136324293911
396 0.00013242062414065003
397 0.00012944739137310535
398 0.0001265510218217969
399 0.00012371984485071152
400 0.00012095463898731396
401 0.00011825640103779733
402 0.00011562347935978323
403 0.00011304975487291813
404 0.00011054205242544413
405 0.00010808950173668563
406 0.00010570111044216901
407 0.00010336509149055928
408 0.00010108393325936049
409 9.885318286251277e-05
410 9.667885024100542e-05
411 9.455623512621969e-05
412 9.247762500308454e-05
413 9.045105252880603e-05
414 8.847190474625677e-05
415 8.65369220264256e-05
416 8.465267455903813e-05
417 8.280154725071043e-05
418 8.100082050077617e-05
419 7.92354840086773e-05
420 7.751955126877874e-05
421 7.583678234368563e-05
422 7.419475150527433e-05
423 7.258712139446

### 4.4 PyTorch：控制流和权重共享

作为动态图和权重共享的一个例子，我们实现了一个非常奇怪的模型：一个全连接的ReLU网络，在每一次前向传播时，它的隐藏层的层数为随机1到4之间的数，这样可以多次重用相同的权重来计算。

因为这个模型可以使用普通的Python流控制来实现循环，并且我们可以通过在定义转发时多次重用同一个模块来实现最内层之间的权重共享。

我们利用Mudule的子类很容易实现这个模型：

In [16]:
import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        在构造函数中，我们构造了三个nn.Linear实例，它们将在前向传播时被使用。
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)

    def forward(self, x):
        """
        对于模型的前向传播，我们随机选择0、1、2、3，
        并重用了多次计算隐藏层的middle_linear模块。
        由于每个前向传播构建一个动态计算图，
        我们可以在定义模型的前向传播时使用常规Python控制流运算符，如循环或条件语句。
        在这里，我们还看到，在定义计算图形时多次重用同一个模块是完全安全的。
        这是Lua Torch的一大改进，因为Lua Torch中每个模块只能使用一次。
        """
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred


# N是批大小；D是输入维度
# H是隐藏层维度；D_out是输出维度
N, D_in, H, D_out = 64, 1000, 100, 10

# 产生输入和输出随机张量
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# 实例化上面定义的类来构造我们的模型
model = DynamicNet(D_in, H, D_out)

# 构造我们的损失函数（loss function）和优化器（Optimizer）。
# 用平凡的随机梯度下降训练这个奇怪的模型是困难的，所以我们使用了momentum方法。
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
    
    # 前向传播：通过向模型传入x计算预测的y。
    y_pred = model(x)

    # 计算并打印损失
    loss = criterion(y_pred, y)
    print(t, loss.item())

    # 清零梯度，反向传播，更新权重 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 680.7816162109375
1 630.7862548828125
2 549.5535278320312
3 458.569580078125
4 671.998046875
5 665.6329345703125
6 655.2793579101562
7 663.2943115234375
8 661.8322143554688
9 206.7633056640625
10 658.6505737304688
11 165.71192932128906
12 649.8790893554688
13 114.63740539550781
14 634.9052734375
15 74.91837310791016
16 62.77189636230469
17 53.311431884765625
18 639.6590576171875
19 521.78369140625
20 39.845420837402344
21 34.85743713378906
22 617.0875244140625
23 547.7255859375
24 32.07247543334961
25 411.61688232421875
26 571.4992065429688
27 40.47376251220703
28 334.9046325683594
29 512.498046875
30 409.3182067871094
31 453.593994140625
32 352.4639892578125
33 63.956077575683594
34 376.20819091796875
35 204.8086395263672
36 58.789886474609375
37 170.0589599609375
38 314.1960144042969
39 128.24288940429688
40 43.43756866455078
41 213.11346435546875
42 91.86834716796875
43 214.34933471679688
44 192.88876342773438
45 74.14707946777344
46 214.47244262695312
47 83.89066314697266
48 173.

388 0.5585256218910217
389 1.6998181343078613
390 0.23382191359996796
391 1.2869199514389038
392 0.5813594460487366
393 0.6555507183074951
394 1.2535728216171265
395 0.4015721082687378
396 0.3067529797554016
397 1.1714823246002197
398 0.7427337169647217
399 0.9647999405860901
400 0.7425968647003174
401 0.10836903750896454
402 0.9651899933815002
403 1.0065391063690186
404 0.6586201190948486
405 0.32355886697769165
406 0.8078217506408691
407 0.3770918846130371
408 0.24031811952590942
409 0.703370988368988
410 0.32322198152542114
411 0.6499772667884827
412 0.4707052409648895
413 0.912745475769043
414 0.6550976634025574
415 0.21263077855110168
416 0.3131241798400879
417 0.19387218356132507
418 0.6160545945167542
419 1.9163745641708374
420 0.9198716878890991
421 0.7541821599006653
422 0.2914096713066101
423 1.4784389734268188
424 0.14821240305900574
425 0.48236483335494995
426 0.6991012692451477
427 0.8967151641845703
428 1.1588819026947021
429 0.41548293828964233
430 0.3890591263771057
431